# Compute code statistics history for NWB repositories

**This notebook requires:**

 * CLOC command-line tool must be installed
 * GitPython (pip install GitPython)
 * matplotlib, numpy, pandas, yaml
 
**This notebook computes:**

 * CLOC stats for the last commit on each day for all NWB code repositories
 * Plots the results for all repos
 
**NOTE:** Computing these statistics is done by checking out all repositorires and then iterating over all commits in a repo, checking out the repo for the last commit on each day, and then computing CLOC. As such, computing these statistics is time consuming. The results can be cached to YAML for further processing and to save time when rerunning and editing the notebook. 

In [ ]:
import os 
import shutil
import subprocess
import git
import time
import yaml
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import pandas as pd
%matplotlib inline

def clean_outdirs(output_dir, source_dir):
    """
    Delete the output directory and all its contents and create a new clean directory.
    
    :returns: A tuple of two strings with the output_dir and source_dir for git sources
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.mkdir(output_dir)
    os.mkdir(source_dir)
    return output_dir, source_dir

def clone_repos(repos, source_dir):
    """
    Clone all of the given repositories. 
    
    :param repos: Dict where the keys are the names of the repos and the
                  values are the git source path to clone
    :param source_dir: Directory where all the git repos should be cloned to.
                  Each repo will be cloned into a subdirectory in source_dir
                  that is named after the corresponding key in the repos dict.
    :returns: Dict where the keys are the same as in repos but the values
              are instances of git.repo.base.Repo pointing to the corresponding
              git repository.
    """
    git_repos = {}
    for k, v in repos.items():
        print("Cloning: %s" % k)
        git_repos[k] = git.Repo.clone_from(v, os.path.join(source_dir, k))
    return git_repos

def run_cloc(cloc_path, src_dir, out_file):
    """
    Run CLOC on the given srcdir, save the results to outdir, and return the parsed
    results.
    """
    command = "%s --yaml --report-file=%s %s" % (cloc_path, out_file, src_dir)
    os.system(command)
    with open(out_file) as f:
        res = yaml.load(f, Loader=yaml.FullLoader)
    return res

def git_repo_stats(repo, cloc_path, output_dir):
    """
    :param repo: The git repository to process
    :type repo: git.repo.base.Repo
    
    :returns: List of dicts with information about all commits. The list
              is sorted in time from most current [0] to oldest [-1]
    """
    # Get hexsha and data of all commits
    commit_stats = []
    # Commits are sorted in time from newest to oldest
    for commit in repo.iter_commits():
        commit_stats.append(
            {'time': time.asctime(time.gmtime(commit.committed_date)),
             'hexsha': commit.hexsha,
             'author': commit.author.name,
             'committer': commit.committer.name,
             'summary': commit.summary,
             'commit': commit})
    # iterate through all the commits in order and compute the cloc stats
    cloc_stats = []
    for commit in commit_stats:
        date = d = time.strftime("%d %b %Y", time.gmtime(commit['commit'].committed_date))
        # Run cloc only for the last commit on each day
        if len(cloc_stats) == 0 or d != cloc_stats[-1]['date']:
            cloc_res = {'hexsha': commit['hexsha'], 'date': date, 'time': commit['time']}
            repo.git.checkout(commit['hexsha'])
            cloc_yaml = os.path.join(
                output_dir, 
                "%s.yaml" % os.path.basename(repo.working_dir))
                #"%s_%s.yaml" % (os.path.dirname(repo.working_dir), commit['hexsha']))
            cloc_res['cloc'] = run_cloc(
                cloc_path=cloc_path, 
                src_dir=repo.working_dir, 
                out_file=cloc_yaml)
            os.remove(cloc_yaml) # Remove the yaml file, we don't need it
            cloc_stats.append(cloc_res)
    return commit_stats, cloc_stats

## Define script settings

In [ ]:
cloc_path = "/Users/oruebel/Devel/Libraries/cloc/cloc"
git_paths = {'PyNWB': "https://github.com/NeurodataWithoutBorders/pynwb.git", 
             'MatNWB': "https://github.com/NeurodataWithoutBorders/matnwb.git",
             'NWBWidgets': "https://github.com/NeurodataWithoutBorders/nwb-jupyter-widgets.git",
             # 'Hackathons': "https://github.com/NeurodataWithoutBorders/nwb_hackathons.git",
             'NWB_Schema': "https://github.com/NeurodataWithoutBorders/nwb-schema.git",
             'NWB_Schema_Language': "https://github.com/NeurodataWithoutBorders/nwb-schema-language.git",
             'HDMF': 'https://github.com/hdmf-dev/hdmf.git',
             'HDMF_Common_Schema': 'https://github.com/hdmf-dev/hdmf-common-schema.git',
             'HDMF_DocUtils': 'https://github.com/hdmf-dev/hdmf-docutils.git',
             # 'HDMF Schema Language' : https://github.com/hdmf-dev/hdmf-schema-language
             'NDX_Template': 'https://github.com/nwb-extensions/ndx-template.git',
             'NDX_Staged_Extensions': 'https://github.com/nwb-extensions/staged-extensions.git',
             #'NDX Webservices': 'https://github.com/nwb-extensions/nwb-extensions-webservices.git',
             'NDX_Catalog': 'https://github.com/nwb-extensions/nwb-extensions.github.io.git',
             #'NDX Extension Smithy': 'https://github.com/nwb-extensions/nwb-extensions-smithy'
             'NWB_1.x_Matlab': 'https://github.com/NeurodataWithoutBorders/api-matlab.git',
             'NWB_1.x_Python': 'https://github.com/NeurodataWithoutBorders/api-python.git'
            }
output_dir = os.path.join(os.getcwd(), 'temp_cloc_stats')
source_dir = os.path.join(output_dir, 'src')
cache_file = os.path.join(os.path.join(os.getcwd(), 'temp_cloc_stats'), 'cloc_stats.yaml')
load_cached_results = True  # load the cloc results from yaml
cache_results = True  # save the cloc results to yaml
show_NWB1 = False  # Remove NWB1 repos from the plots
save_figs = True   # Save the plots to file
# Set all values before this date to 0 for HDMF 2019-03-13 coincides with the removal of HDMF from PyNWB with PR #850
# and the release of HDMF 1.0. For the plotting 2019-03-13 is therefore a good date to start considering HDMF
# stats to avoid duplication of code in statistics, even though the HDMF repo existed on GitHub already since
# 2019-01-23T23:48:27Z, which could be alternatively considered as the start date. Older dates will include
# code history carried over from PyNWB to HDMF. Set to None to consider the full history of HMDF but as mentioned,
# this will lead to some duplicate counting of code before 2019-03-13
hdmf_start_date = '2019-03-13'  
# date when to declare the NWB 1.0 APIs as deprecated. The 3rd Hackathon was held on July 31 to August 1, 2017 at
# Janelia Farm, in Ashburn, Virginia, which marks the date when NWB 2.0 was officially accepted as the 
# follow-up to NWB 1.0. NWB 1.0 as a project ended about 1 year before that.
nwb1_depration_date = '2016-08-01'

## Compute the code statistics for all repos

In [ ]:
# Load cached results if available
if load_cached_results and os.path.exists(cache_file):
    print("Loading cached results: %s" % cache_file)
    with open(cache_file) as f:
        cloc_stats = yaml.load(f, Loader=yaml.FullLoader)
# Compute the results if not cached
else: 
    # Clean and create output directory
    output_dir, source_dir = clean_outdirs()
    # Clone all repos
    print("Cloning all repos...")
    git_repos = clone_repos(repos=git_paths, source_dir=source_dir)
    # Compute CLOC and Commit statistics for all repos
    commit_stats = {}
    cloc_stats = {}
    for name, repo in git_repos.items():
        print("Compute CLOC stats: %s" % name)
        commit_res, cloc_res = git_repo_stats(
            repo, 
            cloc_path=cloc_path, 
            output_dir=output_dir)
        commit_stats[name] = commit_res
        cloc_stats[name] = cloc_res
    # Cache the results if requested
    if cache_results:
        print("Caching results: %s" % cache_file)
        with open(cache_file, 'w') as outfile:
            yaml.dump(cloc_stats, outfile),

## Align and expand results from all repos

Here we create a continoues date range and expand the results from all repos to align with our common time axis. For dates where no new CLOC stats are recorded for a repo, the statistics from the previous time are carried forward to fill in the gaps.

In [ ]:
# Define our reference date range depending on whether we include NWB 1 in the plots or not
if show_NWB1:
    date_range = pd.date_range(start=cloc_stats['NWB_1.x_Matlab'][-1]['date'], end=time.strftime("%d %b %Y", time.localtime()), freq="D")
else:
    date_range = pd.date_range(start=cloc_stats['PyNWB'][-1]['date'], end=time.strftime("%d %b %Y", time.localtime()), freq="D")


# Align and expand our results
repo_sizes_aligned = {}
repo_blanks_aligned = {}
repo_codes_aligned = {}
repo_comments_aligned = {}
repo_nfiles_aligned = {}

for k, v in cloc_stats.items():
    # Dates and CLOC size for the current repo
    curr_dates = pd.pandas.DatetimeIndex([cloc_entry['date'] for cloc_entry in v])[::-1]
    curr_sizes = [np.sum([v for k, v in cloc_entry['cloc']['SUM'].items() if k != 'nFiles']) for cloc_entry in v][::-1]
    curr_blanks = [cloc_entry['cloc']['SUM']['blank'] for cloc_entry in v][::-1]
    curr_codes = [cloc_entry['cloc']['SUM']['code'] for cloc_entry in v][::-1]
    curr_comments = [cloc_entry['cloc']['SUM']['comment'] for cloc_entry in v][::-1]
    curr_nfiles = [cloc_entry['cloc']['SUM']['nFiles'] for cloc_entry in v][::-1]
    
    # Expand the data so we carry forward values for dates where the repo has not changed
    curr_index = 0
    curr_val_sizes = 0
    curr_val_blanks = 0
    curr_val_codes = 0
    curr_val_comments = 0
    curr_val_nfiles = 0
    expanded_sizes = []
    expanded_blanks = []
    expanded_codes = []
    expanded_comments = []
    expanded_nfiles = []
    for d in date_range:
        if d == curr_dates[curr_index]:
            curr_val_sizes = curr_sizes[curr_index]
            curr_val_blanks = curr_blanks[curr_index]
            curr_val_codes = curr_codes[curr_index]
            curr_val_comments = curr_comments[curr_index]
            curr_val_nfiles = curr_nfiles[curr_index]
            if curr_index < (len(curr_dates) -1):
                curr_index += 1
        expanded_sizes.append(curr_val_sizes)
        expanded_blanks.append(curr_val_blanks)
        expanded_codes.append(curr_val_codes)
        expanded_comments.append(curr_val_comments)
        expanded_nfiles.append(curr_val_nfiles)
    repo_sizes_aligned[k] = np.asarray(expanded_sizes)
    repo_blanks_aligned[k] = np.asarray(expanded_blanks)
    repo_codes_aligned[k] = np.asarray(expanded_codes)
    repo_comments_aligned[k] = np.asarray(expanded_comments)
    repo_nfiles_aligned[k] = np.asarray(expanded_nfiles)
    
    
# Convert results to Pandas
repo_sizes_algined_df = pd.DataFrame.from_dict(repo_sizes_aligned)
repo_sizes_algined_df.index = date_range

# Clean up results to mark start date of HDMF
if hdmf_start_date is not None:
    # Set all LOC values prior to the given data to 0
    repo_sizes_algined_df['HDMF'][:hdmf_start_date] = 0
# Clean up results to remove NWB 1.0 software from the graph after 1.0 was deprecated
if nwb1_depration_date is not None:
    repo_sizes_algined_df['NWB_1.x_Matlab'][nwb1_depration_date:] = 0
    repo_sizes_algined_df['NWB_1.x_Python'][nwb1_depration_date:] = 0
# Remove NWB 1 from the datafram if necessary
if not show_NWB1:
    repo_sizes_algined_df = repo_sizes_algined_df.drop('NWB_1.x_Matlab', axis=1).drop('NWB_1.x_Python', axis=1)

## Plot the lines of code statistics

In [ ]:
evenly_spaced_interval = np.linspace(0, 1, len(repo_sizes_algined_df.columns))
colors = [cm.tab20(x) for x in evenly_spaced_interval]
colors = [c for i, c in enumerate(colors) if i % 2 == 0] + [c for i, c in enumerate(colors) if i % 2 == 1]
#plot_order = repo_sizes_algined_df.columns[::-1]
plot_order = ['PyNWB', 'HDMF', 'MatNWB',
              'NWB_Schema_Language', 'NWB_Schema', 
              'HDMF_Common_Schema', 'HDMF_DocUtils',
              'NDX_Catalog', 'NDX_Template', 'NDX_Staged_Extensions',
              'NWBWidgets']
if show_NWB1:
    plot_order += ['NWB_1.x_Matlab', 'NWB_1.x_Python']

repo_sizes_algined_df[plot_order].plot.area(
    figsize=(18,10), 
    stacked=True, 
    linewidth=0, 
    fontsize=16, 
    color=colors)
plt.legend(loc=2, prop={'size': 16})
plt.ylabel('Lines of Code (CLOC)', fontsize=16)
plt.grid(color='black', linestyle='--', linewidth=0.7, axis='both')
plt.title('NWB Code Repository Sizes', fontsize=20)
plt.tight_layout()
if save_figs:
    plt.savefig(os.path.join(output_dir, 'nwb_repo_sizes_all.pdf'))
plt.show()

In [ ]:
for k in git_paths.keys():
    if not show_NWB1:
        if k in ['NWB_1.x_Matlab', 'NWB_1.x_Python']:
            continue
    curr_df = pd.DataFrame.from_dict({'code': repo_codes_aligned[k], 
                                      'blank': repo_blanks_aligned[k], 
                                      'comment': repo_comments_aligned[k]})
    curr_df.index = date_range
    curr_df.plot.area(
        figsize=(18,10), 
        stacked=True, 
        linewidth=0, 
        fontsize=16)
    plt.legend(loc=2, prop={'size': 16})
    plt.ylabel('Lines of Code (CLOC)', fontsize=16)
    plt.grid(color='black', linestyle='--', linewidth=0.7, axis='both')
    plt.title("Lines of Code: %s" % k, fontsize=20)
    plt.tight_layout()
    if save_figs:
        plt.savefig(os.path.join(output_dir, '%s_loc.pdf' % k))
    plt.show()